In [5]:
import hmftpy as hmf
import numpy as np
import matplotlib.pyplot as plt
import hmftpy as hmf
from hmftpy.plaquettes.square import plaq4
from hmftpy.plaquettes.triangular import plaq12
from quspin.basis import spin_basis_1d
from tqdm import tqdm

First off, let's just diagonalize the AF $J_1$ XY model on a square lattice.

In [11]:
basis = spin_basis_1d(4, pauli=0) # pauli=0 fixes the operators as spin-1/2, rather than pauli matrices
J1 = -1
interactions = {'nearest': {'xx': -J1, 'yy': -J1}}
Hi = hmf.operators.inner_hamiltonian(plaq4, interactions, basis)
e, v = Hi.eigh()
ei = e[0]
Hp = hmf.operators.periodic_hamiltonian(plaq4, interactions, basis)
e, v = Hp.eigh()
ep = e[0]
e_hmft, v, mf, cvg = hmf.do_hmft(plaq4, interactions, basis)
print('ED energy with OBC: {}'.format(ei))
print('ED energy with PBC: {}'.format(ep))
print('HMFT energy: {}'.format(e_hmft))
print('HMFT converged? {}'.format(cvg))

ED energy with OBC: -2.828427124746183
ED energy with PBC: -5.656854249492366
HMFT energy: -4.109054536369159
HMFT converged? True


Now, let's add some disorder. How about we turn off interactions randomly. To do this, I will construct a symmetric matrix of 1s and 0s of size L by L, by first constructing a matrix of random floats in this distribution, adding its conjugate, dividing by two, and then rounding to either 0 or 1.

Naively, I expect this to give something around half the initial energy, since I'm removing half the terms from the Hamiltonian. To see if that's the case, I'll average over 20 attempts.

Also, to check my matrices are ok, I'll look at the average of the sum of all entries of the matrices. It should be close to 8.

In [22]:
energies = np.zeros(20)
rs = []
rs_sum = np.zeros(20)
for i in tqdm(range(20)):
    r = np.random.rand(4,4)
    r = np.round((r + r.T)/2) + .00000000001
    rs += [r]
    rs_sum[i] = np.sum(r)
    disorder = {'nearest': {'xx': r, 'yy': r}}
    Hi = hmf.operators.inner_hamiltonian(plaq4, interactions, basis, disorder=disorder)
    e, v = Hi.eigh()
    energies[i] = e[0]
print('Average of the sum of entries in the matrices')
print(np.mean(rs_sum))
print('First matrix')
print(rs[0])
print('Average energy (OBC)')
print(np.mean(energies))
print('Compared to half of no-disorder energy')
print(ei/2)

100%|██████████| 20/20 [00:00<00:00, 60.53it/s]

Average of the sum of entries in the matrices
7.35000000016
First matrix
[[1.e-11 1.e+00 1.e+00 1.e-11]
 [1.e+00 1.e+00 1.e+00 1.e-11]
 [1.e+00 1.e+00 1.e-11 1.e-11]
 [1.e-11 1.e-11 1.e-11 1.e-11]]
Average energy (OBC)
-1.4514242570804181
Compared to half of no-disorder energy
-1.4142135623730916


Now, let's try the same thing with HMFT. I'll reuse each matrix so that won't change anything.

In [23]:
energies_hmft = np.zeros(20)
e_hmft,v, mf,cvg = hmf.do_hmft(plaq4, interactions, basis, disorder=disorder)
cvgs = [False for i in range(20)]
for i, r in enumerate(rs):
    disorder = {'nearest': {'xx': r, 'yy': r}}
    energies_hmft[i], v, mf, cvgs[i] = hmf.do_hmft(plaq4, interactions, basis, disorder=disorder)
print('Average energy (HMFT)')
print(np.mean(energies_hmft))
print('Compared to half of no-disorder energy')
print(e_hmft/2)

Average energy (HMFT)
-1.9051161247134307
Compared to half of no-disorder energy
-1.0422826626538397
